# B-Splines and Trigonmometric Functions

## Sine

The alternating sum of the integer shifts of the first derivative of a
B-spline approaches the sine function as the degree tends to infinity.
More precisely, it holds that

$$\sin(\nu\,\pi)=\lim_{n\rightarrow\infty}\left(-\frac{1}{4}\,\left(\frac{\pi}{2}\right)^{n}
\,\sum_{k\in{\mathbb{Z}}}\,\left(-1\right)^{k}\,\dot{\beta}^{n}(\nu-k)\right).$$
Verify this property visually over the single period $\nu\in[-1,1]$. In the top plot, we
show the ground-truth sine in thin green, and the spline approximation in thicker blue.
In the bottom plot, we show in thicker blue the difference between the sine function and
the spline approximation.

In [49]:
###
# Sine, explicit version
###

# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_sin_plot (
    degree = 3
):
    # Number of samples
    pts = 200 + 1
    # Location of the samples
    abscissa = np.array(
        [-1 + 2 * k / (pts - 1) for k in range(pts)],
        dtype = float
    )
    # Trigonometric data
    sin_data = np.array(
        [math.sin(nu * math.pi) for nu in abscissa],
        dtype = float
    )
    # B-spline explicit approximation of a sine
    normalization = -((math.pi / 2) ** degree) / 4
    spline_data = np.array(
        [
            normalization * math.fsum(
                ((-1) ** k) * sk.grad_b_spline(nu - k, degree)
                for k in range(
                    math.ceil(nu - (degree + 1) / 2),
                    math.floor(nu + (degree + 1) / 2) + 1
                )
            )
            for nu in abscissa
        ],
        dtype = float
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-1.05, 1.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, sin_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Difference plot
    plt.subplot(212)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-0.05, 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-1, 1], [0, 0], "-C2", linewidth = 0.5)
    plt.plot(abscissa, sin_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(b_spline_as_sin_plot, degree = (1, 10))

interactive(children=(IntSlider(value=3, description='degree', max=10, min=1), Output()), _dom_classes=('widge…

## Cosine

The alternating sum of the half-integer shifts of the first derivative of a
B-spline approaches the cosine function as the degree tends to infinity.
More precisely, it holds that

$$\cos(\nu\,\pi)=\lim_{n\rightarrow\infty}\left(-\frac{1}{4}\,\left(\frac{\pi}{2}\right)^{n}
\,\sum_{k\in{\mathbb{Z}}}\,\left(-1\right)^{k}\,\dot{\beta}^{n}(\nu+\frac{1}{2}-k)\right).$$
Verify this property visually over the single period $\nu\in[-1,1]$. In the top plot, we
show the ground-truth cosine in thin green, and the spline approximation in thicker blue.
In the bottom plot, we show in thicker blue the difference between the cosine function and
the spline approximation.

In [50]:
###
# Cosine, PeriodicSpline1D version
###

# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_cos_plot (
    degree = 3
):
    # Number of samples
    pts = 200 + 1
    # Location of the samples
    abscissa = np.array(
        [-1 + 2 * k / (pts - 1) for k in range(pts)],
        dtype = float
    )
    # Trigonometric data
    cos_data = np.array(
        [math.cos(nu * math.pi) for nu in abscissa],
        dtype = float
    )
    # B-spline high-level approximation of a cosine
    normalization = -((math.pi / 2) ** degree) / 4
    spline = sk.PeriodicSpline1D.from_spline_coeff(
        np.array([normalization, -normalization], dtype = float),
        degree = degree,
        delay = -0.5
    ).gradient()
    spline_data = np.append(
        spline.get_samples(-1, support_length = 2, oversampling = (pts - 1) // 2),
        spline.at(1)
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-1.05, 1.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, cos_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Difference plot
    plt.subplot(212)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-0.05, 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-1, 1], [0, 0], "-C2", linewidth = 0.5)
    plt.plot(abscissa, cos_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(b_spline_as_cos_plot, degree = (1, 10))

interactive(children=(IntSlider(value=3, description='degree', max=10, min=1), Output()), _dom_classes=('widge…

## PeriodicSpline1D Class

To compute the spline approximation of the sine function, we have given
above an explicit approach that relies on a sum of several evaluations of
the ``sk.grad_b_spline`` function.  Conversely, to compute the spline
approximation of the cosine function, we have given above an approach
that relies on the higher-level ``sk.PeriodSpline1D`` class.

*   As first exercise, we propose that one computes the spline approximation
    of the sine by relying on the high-level class. (Hint: it is as simple
    as commenting out a single line of the spline approximation of the cosine.)
*   As second exercise, we propose that one computes the spline approximation of
    the cosine by relying on an explicit approach. The second exercise is more
    difficult that the first one.

We give below the solution for each exercise.

In [ ]:
# Sine, PeriodicSpline1D version

# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_sin_plot (
    degree = 3
):
    # Number of samples
    pts = 200 + 1
    # Location of the samples
    abscissa = np.array(
        [-1 + 2 * k / (pts - 1) for k in range(pts)],
        dtype = float
    )
    # Trigonometric data
    sin_data = np.array(
        [math.sin(nu * math.pi) for nu in abscissa],
        dtype = float
    )
    # B-spline high-level approximation of a sine
    normalization = -((math.pi / 2) ** degree) / 4
    spline = sk.PeriodicSpline1D.from_spline_coeff(
        np.array([normalization, -normalization], dtype = float),
        degree = degree
# The default value is delay = 0.0
    ).gradient()
    spline_data = np.append(
        spline.get_samples(-1, support_length = 2, oversampling = (pts - 1) // 2),
        spline.at(1)
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-1.05, 1.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, sin_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Difference plot
    plt.subplot(212)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-0.05, 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-1, 1], [0, 0], "-C2", linewidth = 0.5)
    plt.plot(abscissa, sin_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(b_spline_as_sin_plot, degree = (1, 10))

In [ ]:
# Cosine, explicit version

# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def b_spline_as_cos_plot (
    degree = 3
):
    # Number of samples
    pts = 200 + 1
    # Location of the samples
    abscissa = np.array(
        [-1 + 2 * k / (pts - 1) for k in range(pts)],
        dtype = float
    )
    # Trigonometric data
    cos_data = np.array(
        [math.cos(nu * math.pi) for nu in abscissa],
        dtype = float
    )
    # B-spline explicit approximation of a cosine
    normalization = -((math.pi / 2) ** degree) / 4
    spline_data = np.array(
        [
            normalization * math.fsum(
                # Introduce a delay
                ((-1) ** k) * sk.grad_b_spline(nu + 1 / 2 - k, degree)
                for k in range(
                    # Update the summation bounds
                    math.ceil(nu - degree / 2),
                    math.floor(nu + degree / 2) + 2
                )
            )
            for nu in abscissa
        ],
        dtype = float
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-1.05, 1.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, cos_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Difference plot
    plt.subplot(212)
    plt.xlim(-1.025, 1.025)
    plt.ylim(-0.05, 0.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-1, 1], [0, 0], "-C2", linewidth = 0.5)
    plt.plot(abscissa, cos_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(b_spline_as_cos_plot, degree = (1, 10))